In [5]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
#@title Класс rbc_parser { display-mode: "form" }
#@markdown Чтобы посмотреть код: ПКМ > Форма > Показать код

import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime, timedelta
from IPython import display

class rbc_parser:
    def __init__(self):
        pass
    
    
    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями
        """
        url = 'https://www.rbc.ru/v10/search/ajax/?\
        project={0}&\
        category={1}&\
        dateFrom={2}&\
        dateTo={3}&\
        offset={4}&\
        limit={5}&\
        query={6}&\
        material={7}'.format(param_dict['project'],
                            param_dict['category'],
                            param_dict['dateFrom'],
                            param_dict['dateTo'],
                            param_dict['offset'],
                            param_dict['limit'],
                            param_dict['query'],
                            param_dict['material'])
        
        return url
    
    
    def _get_search_table(self, param_dict: dict,
                          includeText: bool = True) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        
        includeText: bool
        ### Если True, статьи возвращаются с текстами
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        # soup = bs(request.text)
        search_table = pd.DataFrame(r.json()['items'])
        if includeText and not search_table.empty:
            get_text = lambda x: self._get_article_data(x['fronturl'])
            search_table[['overview', 'text']] = search_table.apply(get_text,
                                                                    axis=1).tolist()
            
        return search_table
    
    
    def _get_article_data(self, url: str):
        """
        Возвращает описание и текст статьи по ссылке
        """
        r = rq.get(url)
        soup = bs(r.text, features="lxml") # features="lxml" чтобы не было warning
        div_overview = soup.find('div', {'class': 'article__text__overview'})
        if div_overview:
            overview = div_overview.text.replace('<br />','\n').strip()
        else:
            overview = None
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))
        else:
            text = None
        
        return overview, text 
    
    def get_articles(self,
                     param_dict,
                     time_step = 7,
                     save_every = 5,
                     out: pd.DataFrame = None) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса 
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%d.%m.%Y')
        dateTo = datetime.strptime(param_copy['dateTo'], '%d.%m.%Y')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')
        
        # global out ||| не знаю как out сделать
        if out is None:
            out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime("%d.%m.%Y")
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime("%d.%m.%Y")
            print('Parsing articles from ' + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime("%d.%m.%Y")
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("checkpoint_table.xlsx")
                print('Checkpoint saved')
                save_counter = 0
            
        out.to_excel("table_of_articles_{}.xlsx".format(
            datetime.strptime(param_dict['dateFrom'],
                              '%d.%m.%Y').strftime('%Y')))
        print('Finish')
        # res.to_csv("output.csv", sep = ';')
        
        return out
            


In [3]:
#@title Класс lentaRu_parser { display-mode: "form" }
#@markdown Чтобы посмотреть код: ПКМ > Форма > Показать код 
#@markdown или двойной клик ЛКМ по полю

import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime, timedelta
from IPython import display


class lentaRu_parser:
    def __init__(self):
        pass
    
    
    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])
        
        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])
        
        return search_table

    
    def get_articles(self,
                     param_dict,
                     time_step = 37,
                     save_every = 5,
                     out: pd.DataFrame = None) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса 
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')
        
        # global out ||| не знаю как out сделать
        if out is None:
            out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print('Parsing articles from '\
                  + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("checkpoint_table.xlsx")
                print('Checkpoint saved')
                save_counter = 0
            
        out.to_excel("table_of_articles_{}.xlsx".format(
            datetime.strptime(param_dict['dateFrom'],
                              '%Y-%m-%d').strftime('%Y')))
        print('Finish')
        # res.to_csv("output.csv", sep = ';')
        
        return out

In [42]:
# Импорт диска
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Проект 94'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Проект 94


In [30]:
#@title # Параметры выгрузки { display-mode: "form" }

use_parser = "\u0420\u0411\u041A" #@param ["РБК", "LentaRu"]

#@markdown ## РБК
#@markdown query - поисковой запрос (ключевое слово) \\
#@markdown project - проекты РБК \\
#@markdown category - рубрики \\
#@markdown material - материалы \\
#@markdown dateFrom - с даты \\
#@markdown dateTo - по дату \\
#@markdown offset - смещение поисковой выдачи \\
#@markdown limit - лимит запроса, максимум 100 \\
#@markdown #### Чтобы не специфировать параметр, оставляем поле пустым

query = '\u0420\u0411\u041A'                   #@param {type: "string"}
project = "rbcnews" #@param ["rbcnews", "rbctv", "rbcstyle", "sport", "realty", "crypto", "autonews", "quote", "bc3", "trends"] {allow-input: true}
category = "TopRbcRu_economics" #@param ["TopRbcRu_economics", "TopRbcRu_auto", "TopRbcRu_business", "TopRbcRu_money", "TopRbcRu_realty", "TopRbcRu_society", "TopRbcRu_politics", "TopRbcRu_own_business", "TopRbcRu_specials", "TopRbcRu_technology_and_media", "TopRbcRu_finances"] {allow-input: true}
material = "" #@param ["video", "quiz", "interview", "research", "card", "opinion", "multimedia", "short_news", "olympics_online", "online", "investigation", "rating", "article_specproject", "article", "story"] {allow-input: true}
dateFrom = '2000-01-01'         #@param {type: "date"}
dateTo = "2000-12-31"           #@param {type:"date"}
offset = 0                      #@param {type:"slider", min:0, max:100, step:1}
limit = 100 #@param {type:"slider", min:0, max:100, step:1}

#@markdown ---

if use_parser == "РБК":
    param_dict = {'query'   : query, 
                  'project' : project,
                  'category': category,
                  'dateFrom': datetime.
                  strptime(dateFrom, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'dateTo'  : datetime.
                  strptime(dateTo, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'offset'  : str(offset),
                  'limit'   : str(limit),
                  'material': material}

#@markdown ## LentaRu
#@markdown query - поисковой запрос (ключевое слово) \\
#@markdown offset - cмещение поисковой выдачи \\
#@markdown size - количество статей. Ограничено время запроса, точного лимита нет. 1000 работает почти всегда \\
#@markdown sort - сортировка по дате: (2) - по убыванию, (3) - по возрастанию; по релевантности (1) \\
#@markdown title_only - точная фраза в заголовке (1) \\
#@markdown domain - ? \\
#@markdown material - материалы: Все материалы (0). Новость (1) \\
#@markdown block - рубрика: Экономика (4). Все рубрики (0) \\
#@markdown dateFrom - с даты \\
#@markdown dateTo - по дату \\
#@markdown #### Чтобы не специфировать параметр, оставляем поле пустым

query = '\u0420\u0411\u041A'                   #@param {type: "string"}
offset = 0 #@param {type:"integer"}
size = 1000 #@param {type:"slider", min:0, max:1000, step:1}
sort = "3" #@param ["1", "2", "3"] {allow-input: true}
title_only = "0" #@param ["0", "1"]
domain = "1" #@param ["1"] {allow-input: true}
material = "0" #@param ["0", "1", "2", "3", "4"] {allow-input: true}
bloc = "4" #@param ["0", "1", "2", "3", "4"] {allow-input: true}
dateFrom = '2000-01-01'         #@param {type: "date"}
dateTo = "2000-12-31"           #@param {type:"date"}

#@markdown ---

if use_parser == "LentaRu":
    param_dict = {'query'     : query, 
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material, 
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)


РБК - param_dict: {'query': 'РБК', 'project': 'rbcnews', 'category': 'TopRbcRu_economics', 'dateFrom': '01.01.2000', 'dateTo': '31.12.2000', 'offset': '0', 'limit': '100', 'material': ''}


In [31]:
# Пример запроса таблицы по параметрам без описания и текста статьи.
# Действует ограничение в 100 статей на 1 запрос (параметром limit)
assert use_parser == "РБК"
parser = rbc_parser()
tbl = parser._get_search_table(param_dict, includeText = False)
tbl.head()

,id,fronturl,publish_date_t,publish_date,title,photo,project,category,opinion_authors,authors,anons
0,5703b2279a7947783a5a19a6,http://www.rbc.ru/economics/28/12/2000/5703b22...,977983443,"Thu, 28 Dec 2000 09:04:03 +0300","Новая станция метро, рожденная в муках",{'url': 'http://pics.v7.top.rbk.ru/v6_top_pics...,None,Экономика,None,None,"... метро, строительство которой началось ещ..."
1,5703b2279a7947783a5a19a7,http://www.rbc.ru/economics/28/12/2000/5703b22...,977982706,"Thu, 28 Dec 2000 08:51:46 +0300",Журналисты Приморья - за Наздратенко горой,{'url': 'http://pics.v7.top.rbk.ru/v6_top_pics...,None,Экономика,None,None,"... только функционирование сигнализации, пр..."
2,5703b2269a7947783a5a1992,http://www.rbc.ru/economics/26/12/2000/5703b22...,977840096,"Tue, 26 Dec 2000 17:14:56 +0300",Москвичи могут спать спокойно,{'url': 'http://pics.v7.top.rbk.ru/v6_top_pics...,None,Экономика,None,None,... обращать на проверку припаркованных рядо...
3,5703b2259a7947783a5a1976,http://www.rbc.ru/economics/23/12/2000/5703b22...,977550766,"Sat, 23 Dec 2000 08:52:46 +0300",Юлию Тимошенко вызвали в Генпрокуратуру РФ,{'url': 'http://pics.v7.top.rbk.ru/v6_top_pics...,None,Экономика,None,None,"... ведомств. Не исключено, что допрос состо..."
4,5703b2259a7947783a5a1978,http://www.rbc.ru/economics/22/12/2000/5703b22...,977504869,"Fri, 22 Dec 2000 20:07:49 +0300",Явлинский пытается спасти свою партию,{'url': 'http://pics.v7.top.rbk.ru/v6_top_pics...,None,Экономика,None,None,"... "" решения и полном переизбрании руководст..."


In [32]:
# Собирает новости итеративно
table = parser.get_articles(param_dict=param_dict)

Checkpoint saved
Parsing articles from 26.12.2000 to 31.12.2000
Finish


In [ ]:
# Тоже будем собирать итеративно, правда можно ставить time_step побольше, т.к.
# можно ставить больше лимит на запрос статей
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict)
print(len(tbl.index))
tbl.head()

Checkpoint saved
Parsing articles from 2020-10-22 to 2020-11-01
Finish
97578


,rightcol,docid,image_url,lastmodtime,part,title,type,url,tags,bloc,domain,modified,text,pubdate,status,snippet
0,В минувшее воскресенье исполняющий обязанности...,407657,https://icdn.lenta.ru/images/0000/0257/0000025...,1499945319,0,"Дорожает алкоголь, табак, бензин, автомобили и...",1,https://lenta.ru/news/2000/01/03/aktsizy/,[],4,1,946902825,В минувшее воскресенье исполняющий обязанности...,946902825,0,"В минувшее воскресенье исполняющий ... ""Об акц..."
1,Центробанк хочет усилить госкотроль за валютны...,407798,https://icdn.lenta.ru/images/0000/0257/0000025...,1499945324,0,Виктор Геращенко предлагет усилить госконтроль...,1,https://lenta.ru/news/2000/01/05/currency/,[],4,1,947084781,Усилить госконтроль на валютном рынке предложи...,947084781,0,Усилить госконтроль на валютном рынке ... Цент...
2,Началась промышленная печать монет и купюр ЕВРО,407746,https://icdn.lenta.ru/images/0000/0257/0000025...,1499945322,0,Европа печатает и чеканит новые деньги,1,https://lenta.ru/news/2000/01/05/evro/,[],4,1,947103402,В Европе началась промышленная печать купюр и ...,947103402,0,В Европе началась промышленная печать купюр и ...
3,Цена российской нефти снизилась,407918,https://icdn.lenta.ru/images/0000/0257/0000025...,1499945329,0,Российская нефть дешевеет,1,https://lenta.ru/news/2000/01/06/oil/,[],4,1,947163766,Цена российской нефти Urals с доставкой в порт...,947163766,0,Цена российской нефти Urals с доставкой ... со...
4,Доступ нефтедобывающих компаний к экспортной т...,407888,https://icdn.lenta.ru/images/0000/0257/0000025...,1499945327,0,Нефтедобывающим компаниям-должникам ограничат ...,1,https://lenta.ru/news/2000/01/06/export/,[],4,1,947172661,В 2000 году доступ нефтедобывающих компаний к ...,947172661,0,В 2000 году доступ нефтедобывающих ... недоимк...
